In [52]:
import mne
import os.path as op
from stormdb.access import Query
from sys import argv
import pickle
import numpy as np

In [15]:
proj_name = 'MINDLAB2020_MEG-AuditoryPatternRecognition'
wdir = '/projects/' + proj_name + '/scratch/working_memory/'
data_dir = wdir + 'averages/data/'
mode = 'patterns'
suffix = 'task_sensor_lf_0.05_hf_None_tstep_0.025_twin_0.05'
qr = Query(proj_name)
subs = qr.get_subjects()
scode = 11
if len(argv) > 1:
    scode = int(argv[1])
sub = subs[scode-1] 

In [28]:
cfname = op.join(data_dir,sub,sub + '_patterns_' + suffix + '.p')
times_fname = op.join(data_dir,sub,sub + '_times_' + suffix + '.p')
inv_fname = op.join(data_dir,sub,sub + '_evoked_inverse_lf_0.05_hf_40_tstep_0.025_twin_0.05.p')

cfile = open(cfname,'rb')
p = pickle.load(cfile)
cfile.close()

times_file = open(times_fname,'rb')
times = pickle.load(times_file)
times_file.close()

inv_file = open(inv_fname,'rb')
inv = pickle.load(inv_file)
inv_file.close()

/tmp/ipykernel_3563/2317333950.py:6: DeprecationWarning: Info does not support directly setting the key 'srate', this warning will turn into an error after 0.24. You can set info['temp'] to store temporary objects in an Info instance, but these will not survive an I/O round-trip.
  p = pickle.load(cfile)
/tmp/ipykernel_3563/2317333950.py:6: DeprecationWarning: Info does not support directly setting the key 'srate', this warning will turn into an error after 0.24. You can set info['temp'] to store temporary objects in an Info instance, but these will not survive an I/O round-trip.
  p = pickle.load(cfile)


In [47]:
for cp in p:
    p[cp].times = times[cp[0:-1]]

In [48]:
p['interaction'] = mne.combine_evoked([p['manipulation1'],p['maintenance1']],weights=[1,-1])

/users/david/miniconda3/envs/mne/lib/python3.9/copy.py:153: DeprecationWarning: Info does not support directly setting the key 'srate', this warning will turn into an error after 0.24. You can set info['temp'] to store temporary objects in an Info instance, but these will not survive an I/O round-trip.
  y = copier(memo)


In [49]:
sources = {}
for e in p:
    sources[e] = mne.beamformer.apply_lcmv(p[e],inv)
    sources[e].tmin = times[cp[0:-1]][0]
    sources[e].tmax = times[cp[0:-1]][-1]
    sources[e].tstep = np.diff(times[cp[0:-1]])[0]

In [50]:
sources

{'maintenance1': <VolSourceEstimate | 12134 vertices, subject : 0011_U7X, tmin : -75.00000000000001 (ms), tmax : 3974.9999999999995 (ms), tstep : 25.0 (ms), data shape : (12134, 163), ~15.2 MB>,
 'manipulation1': <VolSourceEstimate | 12134 vertices, subject : 0011_U7X, tmin : -75.00000000000001 (ms), tmax : 3974.9999999999995 (ms), tstep : 25.0 (ms), data shape : (12134, 163), ~15.2 MB>,
 'interaction': <VolSourceEstimate | 12134 vertices, subject : 0011_U7X, tmin : -75.00000000000001 (ms), tmax : 3974.9999999999995 (ms), tstep : 25.0 (ms), data shape : (12134, 163), ~15.2 MB>}

In [51]:
src_fname = op.join(data_dir,sub,sub + '_{}_sources_{}.p'.format(mode,suffix))
src_file = open(src_fname,'wb')
pickle.dump(sources,src_file)
src_file.close()